<a href="https://colab.research.google.com/github/babypotatotang/Introduction-to-DeepLearning/blob/main/9.%20Word%20Embedding/05.Glove.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

* **GloVe**(Global Vectors for Word Representation)은 카운트 기반과 예측 기반을 모두 사용하는 방법론으로, 기존 카운트 기반의 LSA와 예측 기반의 Word2Vec의 단점을 보완하여 개발되었음. 

# **1. 기존 방법론에 대한 비판**
--- 
* LSA는 각 단어의 빈도수를 카운트한 행렬이라는 전체적인 통계 정보를 입력으로 받아 차원을 축소하여 잠재된 의미를 끌어내는 방법론
* Word2Vec은 실제값과 예측값에 대한 오차를 손실함수를 통해 줄여나가며 학습하는 예측 기반 방법론임. 

### 각 방식의 장, 단점
* LSA는 카운트 기반으로 전체적인 통계 정보를 고려하지만 유추작업에서는 성능이 떨어지고, Word2Vec은 예측 기반으로 유추 작업에서 성능은 뛰어나지만 윈도우 크기 내에서만 주변 단어를 고려하기 때문에 코퍼스의 전체 정보를 반영하지 못함. 

# **2. Window based Co-occurrence Matrix)**
--- 
* 단어의 동시 등장 행렬은 행과 열을 전체 단어 집합의 단어들로 구성하고, i 단어의 윈도우 크기 내에서 k 단어가 등장한 횟수를 i행 k열에 기재한 행렬을 말함. 

### ex)  윈도우 크기가 1일때, 아래 텍스트를 가지고 구성한 동시 등장 행렬은 다음과 같음. 

I like deep learning
I like NLP
I enjoy flying 

|카운트|I|like|enjoy|deep|learning|NLP|flying
|------|---|---|---|---|---|---|---|
|I|0|2|1|0|0|0|0|
|like|2|0|0|1|0|1|0|
|enjoy|1|0|0|0|0|0|1|
|deep|0|1|0|0|1|0|0|
|learning|0|0|0|1|0|0|0|
|NLP|0|1|0|0|0|0|0|
|flying|0|0|1|0|0|0|0|

* 위 행렬은 행렬을 전치해도 동일한 행렬이 된다는 특징이 있음. 

# **3. Co-Occurrence Probability**
---
* 동시 등장 확률 $P(k|i)$는 동시 등장 행렬로부터 특정 단어 i의 전체 등장 횟수를 카운트하고, 득정 단어 i가 등장했을때, 어떤 단어 k가 등장한 횟수를 카운트하여 계산한 조건부 확률
* $P(k|i)$에서 i를 중심단어(Center word), k를 주변단어(Context Word)라고 했을때, 
    * 위에서 배운 동시 등장 행렬에서 중심 단어 i의 행의 모든 값을 더한 값을 분모
    * i행 k열의 값을 분자  

|동시 등장 확률과 크기 관계 비(ratio)|k=solid|k=gas|k=water|k=fasion
|------------------------------------|-------|-----|-------|---------|
|P(k l ice)|0.00019|0.000066|0.003|0.000017|
|P(k l steam)| 0.000022 | 0.00078 | 0.0022 | 0.000018 |
|P(k l ice) / P(k l steam) | 8.9 | 0.085 | 1.36 | 0.96 |

* `ice`가 등장했을때 `solid`가 등장할 확률은 `steam`이 등장했을때 `solid`가 나타날 확률보다 높음.

# **4. 손실함수(Loss Function)**
### 용어 정리 

* $X$ : 동시 등장 행렬(Co-occurrence Matrix)
* $X_{ij}$ : 중심 단어 i가 등장했을 때 윈도우 내 주변 단어 j가 등장하는 횟수
* $X_i$: 동시 등장 행렬에서 i행의 값을 모두 더한 값
* $P_{ik}$: 중심 단어 i가 등장했을 때 윈도우 내 주변 단어 k가 등장할 확률
    - Ex) P(solid l ice) = 단어 ice가 등장했을 때 단어 solid가 등장할 확률
* $\omega_i$: 중심 단어 i의 임베딩 벡터
* $ \widetilde{\omega_k}$: 주변 단어 k의 임베딩 벡터

* GloVe의 아이디어를 한 줄로 요약하면 **'임베딩 된 중심 단어와 주변 단어 벡터의 내적이 전체 코퍼스에서의 동시 등장 확률이 되도록 만드는 것'** 

$$F(\omega_i, ω_j, \widetilde{\omega_k}) =  \frac{P_{ik}}{P_{jk}}$$
* 세 개의 벡터에 어떤 함수 $F$를 수행하면 다음의 확률이 나온다는 초기식으로 부터 전개 시작
* 함수 $F$는 두 단어 사이의 동시 등장 확률의 크기 관계 비(ratio) 정보를 벡터 공간에 인코딩하는 것이 목적이며, 이를 통해 두 벡터의 차이을 $F$의 입력으로 사용하는 것을 제안함. 
$$F((\omega_i-ω_j)^T, \widetilde{\omega_k}) =  \frac{P_{ik}}{P_{jk}}$$
* 하지만 우변은 스칼라값이고 좌변은 벡터 값이기 때문에 선형 공간에서 단어 의미 관계를 표현하고자 내적 연산을 사용함.

* 이때 주의해야할 점은 중심 단어 $\omega$와 $\widetilde{\omega}$는 무작위로 선택되는 것이므로, 이 둘의 관계를 자유롭게 교환할 수 있게 해야함. 이를 만족하기 위해 함수 $F$는 다음의 식을 만족해야함. 
$$F(a+b) = F(a)F(b), \forall a, b \in ℜ$$

* 이때 $a$와 $b$를 각각 기존의 벡터에 적용시킨다면 덧셈이 뺄셈으로 곱셈이 나눗셈으로 바뀌게 됨. 
$$F(v_1^Tv_2-v_3^Tv_4) = \frac{v_1^Tv_2}{v_3^Tv_4}, ∀v_1,v_2,v_3,v_4 \in V$$
$$F((\omega_i - \omega_j)^T \widetilde{\omega_k}) = \frac{\omega_i^T\widetilde{\omega_k}}{\omega_j^T\widetilde{\omega_k}}$$
* 여기서 원래 $F$ 함수의 결과 값은 $$\frac{P_{ik}}{P_{jk}}$$이었음. 
$$\frac{P_{ik}}{P_{jk}} =  \frac{\omega_i^T\widetilde{\omega_k}}{\omega_j^T\widetilde{\omega_k}}$$
이 식을 풀어쓰면 

$$ F(\omega_i^T\widetilde{\omega_k} - \omega_j^T\widetilde{\omega_k}) = \frac{\omega_i^T\widetilde{\omega_k}}{\omega_j^T\widetilde{\omega_k}} $$

* 이를 만족시키는 함수를 지수함수임. 

### 손실함수
$$ Loss Function =  \sum_{m,n = 1}^V (\omega_m^T  \widetilde{\omega_n} + b_m + \widetilde{b_n} - logX_{mn})^2 $$
* 손실함수는 우변의 값과의 차이를 최소화 하는 방향으로 학습하게 됨. 하지만 다음의 문제로 새로운 가중치 함수를 정의 함. 

* 문제 1) $logX_{ik}$가 0이 될 수 있음. 


    -> $log(1+X_{ik})$로 변경 

* 문제 2) DTM 처럼 희소 행렬일 가능성이 다분함. 즉 많은 값이 0이거나 너무 많은 값을 가질 수 있음. 


    ->  이에 대한 가중치를 부여하기로함. 가중치 그래프는 다음과 같음. 
![](https://wikidocs.net/images/page/22885/%EA%B0%80%EC%A4%91%EC%B9%98.PNG)

* $X_{ik}$의 값이 작으면 상대적으로 함수의 값은 작도록 하고, 값이 크면 함수의 값은 상대적으로 크게 설정 이 함수는 다음과 같이 정의 됨. 
$$f(x) = min(1, (x/x_{max})^0.75)$$

* 최종적으로 다음과 같은 일반화 손실함수를 얻을 수 있음. 
$$ Loss Function =  \sum_{m,n = 1}^V f(X_{mn}) (\omega_m^T  \widetilde{\omega_n} + b_m + \widetilde{b_n} - logX_{mn})^2 $$

# **5. GloVe 훈련하기**
---

In [ ]:
! pip install glove-python-binary

In [ ]:
import re
import urllib.request
import zipfile
from lxml import etree
from nltk.tokenize import word_tokenize, sent_tokenize

import nltk
nltk.download('punkt')

In [ ]:
# 데이터 다운로드
urllib.request.urlretrieve("https://raw.githubusercontent.com/ukairia777/tensorflow-nlp-tutorial/main/09.%20Word%20Embedding/dataset/ted_en-20160408.xml", filename="ted_en-20160408.xml")

# 데이터 전처리 
targetXML = open('ted_en-20160408.xml', 'r', encoding='UTF8')
target_text = etree.parse(targetXML)

# xml 파일로부터 <content>와 </content> 사이의 내용만 가져온다.
parse_text = '\n'.join(target_text.xpath('//content/text()'))

# 정규 표현식의 sub 모듈을 통해 content 중간에 등장하는 (Audio), (Laughter) 등의 배경음 부분을 제거.
# 해당 코드는 괄호로 구성된 내용을 제거.
content_text = re.sub(r'\([^)]*\)', '', parse_text)

# 입력 코퍼스에 대해서 NLTK를 이용하여 문장 토큰화를 수행.
sent_text = sent_tokenize(content_text)

# 각 문장에 대해서 구두점을 제거하고, 대문자를 소문자로 변환.
normalized_text = []
for string in sent_text:
     tokens = re.sub(r"[^a-z0-9]+", " ", string.lower())
     normalized_text.append(tokens)

# 각 문장에 대해서 NLTK를 이용하여 단어 토큰화를 수행.
result = [word_tokenize(sentence) for sentence in normalized_text]

In [ ]:
from glove import Corpus, Glove

corpus = Corpus()
corpus.fit(result, window = 5) # 훈련 데이터로부터  Glove에 사용할 동시 등장 행렬 생성

In [ ]:
glove = Glove(no_components=100, learning_rate=0.05)
glove.fit(corpus.matrix, epochs=20, no_threads=4, verbose=True)
glove.add_dictionary(corpus.dictionary)

In [ ]:
model_result1 = glove.most_similar("man")
print(model_result1)

In [ ]:
model_result2 = glove.most_similar('boy')
print(model_result2)